# Bayesian Classifier
In this lab, you will implement and assess the performance of the Bayesian Classifier.

## Lab Instructions:
1. Read the explanation above each requirement very well
2. Read the requirement very well before jumping into the code.
3. Some requirements have essay questions in them, make sure you do NOT miss them.
4. PLEASE Read the hints! They are clear and made to help you complete the requirement as fast as you should 

In [1]:
#### always keep all your imports in the first cell ####
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import csv
import math

%matplotlib qt

## Requirement

In this requirement, you will build the Bayesian Classifier and test its performance. 

You are provided with a data file **data1.csv** containing list of points and their corresponding classes. The format of the data files is shown in the table below.

| |Class|Feature 1|Feature 1| 
|-|-|-|-|
|Point#1|1|0.271633|-2.93224|
|Point#2|1|7.020786|-1.98966|
|Point#3|1|2.901827|-0.91291|


You are also provided with a test data file **test_data.csv**. The file contains test points that are unlabelled (i.e. the class to which they belong is unknown).

In [2]:
# TODO [1] : Read the file 'data1.csv' into the variable data.
# data contains the training data together with labelled classes.
def read_data(file_name):
    ## HINT 1: How is the data ordered in the file?
    ## HINT 2: Do you need to cast the data you read from the file?
    data = np.loadtxt(file_name,delimiter=',')
    
    return data

In [3]:
def read_test_data():
    
    # TODO [2.A]: Read the file 'test_data.csv' into the variable test_data
    # test_data contains the unlabelled test class.
    ## HINT: Do you need to cast the data you read from the file?

    test_data = read_data('test_data.csv')
    
    # TODO [2.B]: Read the file 'test_data_true.csv' into the variable test_data_true
    # test_data_true contains the actual classes of the test instances, which you will compare
    # against your predicted classes.
    ## HINT: Do you need to cast the data you read from the file?

    test_data_true = read_data('test_data_true.csv')
    return test_data, test_data_true

### Machine Learning Terminlology
Machine learning problems use common termonology (names and notiations) to refer to certain things. It is useful to use this termonology throughout your code to make it readable.

| | |
|:-|:--- |
|$M$:|A scalar; represents the number of training points in the training set.|
|$K$:|A scalar; represents the number of test points in the test set.|
|$N$:|A scalar; represents the number of features of training set/test set (dimensionality of data).|
|$X$:|A numpy array of shape $(M \times N)$ containing the training data **without** its labels, where $M$ is the number of training points and $N$ is the number of features in the dataset (or dimensionality of features). <br/> Each element in $X$ is a tuple $(X_1, X_2, \dots, X_N)$ where $N$ is the number of features in the dataset.| 
|$X_{test}$:| A numpy array of shape $(K \times N)$ containing the test data, where $K$ is the number of test points and $N$ is the number of features in the dataset (or dimensionality of features). <br/> Each element in $X_{test}$ is a tuple $(X_1, X_2, \dots, X_N)$ where $N$ is the number of features in the dataset. <br/> The number of columns in $X_{test}$ is equal to the number of columns in $X$ (because they have the same number of features). However, the number of rows in $X_{test}$ is different to the number of rows in $X$.|
|**$Y$:| A numpy array of shape $(M \times 1)$ containing the labels of the training data. Each row in $Y$ corresponds to the label of the training point in $X$.<br/> For example, $Y[j]$ corresponds to the label of the training point $X[j]$ where $0<=j<M$.|



In [9]:
# TODO [3]: Fill the values of M, K, N, X, XTest, and Y respectively.
# Do not fill them manually (i.e. do not set N = 3). They should be generic for any input file.  
training_data = read_data('data1.csv')
test_data, test_data_true = read_test_data()

numClasses = 3 
M = training_data.shape[0]
N = training_data.shape[1] - 1
K = test_data.shape[0]

X = training_data
X_Test = test_data
Y = test_data_true

In [22]:
# TODO [4]: Draw a scatter plot for training data, where each class is coloured by the color corresponding 
#           to its index in the colors array.
# Class 1 should be coloured in red, Class 2 should be coloured in green, and Class 3 should be coloured in blue.
# Hint: We have done a similar plot in the previous lab. What operation do we need to select training data 
#       belonging to a certain class?

colors = ['r', 'g', 'b', 'c', 'y']


for classIndex in range(numClasses):
    class_data = X[X[:,0]==classIndex+1]
    plt.scatter(class_data[:,1],class_data[:,2],c=colors[classIndex],label='Class '+str(classIndex+1))
plt.title('Training Data')
plt.legend()
plt.show()

In [18]:
## What do you notice about the plot? (Write everything you can think of)
'''
    Your Answer:
''' 

'\n    Your Answer:\n'

### Bayesian Classifier
The Bayesian Classifier calculates the probability of the test point belonging to each class, then the class with highest probability is assigned to the test point.

Classification of $x_{test}$ = $argmax_{i} P\big(C_i|x_{test}\big)$ = $argmax_{i} P(x|C_i) * P(C_i)$

* $P(C_i|x_{test})$: Posterior probability
* $P(x|C_i)$: Class-conditional probability (or distribution)
* $P(C_i)$: Class apriori probability
                
**Note that** $P(C_i|x_{test}) \neq P(x_{test}|C_i) * P(C_i)$. Instead,  $P(C_i|x_{test}) = \frac{P(x_{test}|C_i) * P(C_i)}{P(x_{test})}$. However, when we compare multiple classes, the denominator $P(x_{test})$ is independent of the class $i$ and can be regarded as normalizing factor.

**We start by** computing statistical parameters about each class from the data. 

For each class, we are interested in **three** parameters that will be used for calculating the Gaussian class-conditional distribution and the posterior probability.

These parameters are:

|||
|:-|:-|
|**Class Apriori Probability: ($P_C$)**| A scalar; the probability of class occurence (how frequent this class appears in the training data)|
|**Class Mean: ($\mu$)**| A vector of shape $(N \times 1)$, it is the expected value (mean) calculated from the training points of each class.|
|**Class Covariance Matrix: ($\Sigma$)**| A square symmetric matrix of shape $(N \times N)$ representing the covariances between all the feature calculated from the training points of the class. <br/> For example: Matrix element $\sigma^2_{12}$ is the covariance between the 1st and the 2nd features|



In [23]:
# A list of size (numClasses, 1) containing the a priori probabilities of each class in the training set.
pClasses = [None]*numClasses

# A numpy array of size (numClasses, N) containing the mean points of each class in the training set.
estimate_means = [None]*numClasses
                    # HINT: USE NP.MEAN

# A numpy array of size (numClasses, N, N) containing the covariance matrices of each class in the training set.
                          # HINT: USE NP.COV (Pay attenention for what it takes as an argument)
estimate_covariances = [None]*numClasses
                             
for classIndex in range(numClasses):
    # TODO [5]: Estimate the parameters of the Gaussian distributions of the given classes.
    # Fill pClasses, estimate_means, and estimate_covariances in this part 
    # Your code should be vectorized WITHOUT USING A SINGLE FOR LOOP.
    class_data = X[X[:,0]==classIndex+1]
    pClasses[classIndex] = class_data.shape[0]/M
    estimate_means[classIndex] = np.mean(class_data[:,1:],axis=0)
    estimate_covariances[classIndex] = np.cov(class_data[:,1:].T)
    
pClasses=np.array(pClasses)
estimate_means=np.array(estimate_means)
estimate_covariances=np.array(estimate_covariances)

In [29]:
### Test your implementation ###
### DO NOT CHANGE THIS CODE ###
assert len(pClasses) == numClasses,\
        'Incorrect class apriori probability list, it should be of length {}'.format(len(pClasses))
assert 1-np.sum(pClasses)<=10**-9,\
        'Sum of apriori probabilities should be 1, found {}'.format(np.sum(pClasses))

assert estimate_means.shape == (numClasses, N),\
        'Incorrect estimated means, it should be of shape {}'.format((numClasses, N))
assert estimate_covariances.shape == (numClasses, N, N),\
        'Incorrect estimate covariance matrices, it should be of shape {}'.format((numClasses, N, N))

**The second step** in the classifier is to calculate the class-conditional density using the Gaussian destribution:

$P(x|C_i) = \mathcal{N}(x; \mu_i, \Sigma_i) = \frac{1}{(2\pi)^{\frac{N}{2}}|\Sigma_i|^{\frac{1}{2}}} exp\big(\frac{-1}{2}(x-\mu_i)^T\Sigma^{-1}_{i}(x-\mu_i)\big)$

In [128]:
# TODO 6: Implement the multivariate normal gaussian distribution with parameters mu and sigma, and return the
#  value in prob.
def multivariate_normal_gaussian(X, mu, sigma):
    x_mu=X-mu
    if(sigma.ndim!=3):
        sigma=sigma[np.newaxis,:,:]
        x_mu = x_mu[np.newaxis,:,:]
    else :
        x_mu=x_mu[:,:,np.newaxis]
    print(x_mu.shape)
    # print(sigma.shape)
    
    # Using np.linalg.det and np.linalg.inv
    N=X.shape[0]
    correlation_matrix_determinant = np.linalg.det(sigma)
    correlation_matrix_inverse = np.linalg.inv(sigma)
    print(correlation_matrix_inverse.shape)
    denominator = (2*np.pi)**(N/2) * np.sqrt(correlation_matrix_determinant)
    denominator=denominator.reshape(-1,1)
    numerator = np.exp(-0.5 * np.transpose(x_mu,(0,2,1)) @ correlation_matrix_inverse @ x_mu)
    numerator=numerator.reshape(-1,1)
    prob = numerator/denominator
    
    return prob

In [129]:
### Test your implementation ###
### DO NOT CHANGE THIS CODE ###
np.random.seed(90)
assertion_x = np.random.rand(3).reshape(-1,1)
assertion_mu = np.random.rand(3).reshape(-1,1)
assertion_sigma = np.random.rand(9).reshape(3,3)
assertion_probability = multivariate_normal_gaussian(assertion_x, assertion_mu, assertion_sigma)[0][0]
print(assertion_probability)
assertion_probability = round(assertion_probability, 1)
print(assertion_probability)

assert assertion_probability == 7.8,\
    'Incorrect Gaussian Probability calculated'

(1, 3, 1)
(1, 3, 3)
7.806733924973923
7.8


**The final step** is to go for each test point, calculate its posterior probability against each class, then classify it to the class with the highest posterior probability.

In [134]:
# TODO [7]: Apply the Bayesian Classifier to predict the classes of the test points.
predicted_classes = K*[None] # predicted_classes: A numpy array of size (K, 1) where K is the number of points in the test set. Every element in this array
                       # contains the predicted class of Bayes classifier for this test point.

for i in range(X_Test.shape[0]):
    print("For test point:", X_Test[i])
    classProbabilities = multivariate_normal_gaussian(X_Test[i], estimate_means, estimate_covariances)[:,0]
    # TODO [7.A]: Compute the probability that the test point X_Test[i] belongs to each class in numClasses.
    #  Fill the array classProbabilities accordingly.
    # print(X_Test[i].shape)
    # print(estimate_means.shape)
    # print(estimate_covariances.shape)
    
    
    
    # TODO [7.B]: Find the prediction of the test point X_Test[i] and append it to the predicted_classes array.
    predicted_classes[i] = np.argmax(classProbabilities)+1
    print('Class Probabilities: ', classProbabilities)  # the first class is the left most in the scatter plot
    print("Predicted class is :", predicted_classes[i])
    print("******************************************************************************")


For test point: [10.70141426  3.87253627]
(3, 2, 1)
(3, 2, 2)
Class Probabilities:  [6.11034970e-12 6.16974465e-17 1.14599125e-03]
Predicted class is : 3
******************************************************************************
For test point: [-3.81831787 -5.00977814]
(3, 2, 1)
(3, 2, 2)
Class Probabilities:  [3.54332497e-02 4.59234781e-10 8.95192185e-07]
Predicted class is : 1
******************************************************************************
For test point: [-3.57071936  9.96036181]
(3, 2, 1)
(3, 2, 2)
Class Probabilities:  [1.82496320e-19 3.92600735e-03 2.47810130e-10]
Predicted class is : 2
******************************************************************************
For test point: [ 4.94309043 -0.01539406]
(3, 2, 1)
(3, 2, 2)
Class Probabilities:  [4.50268122e-06 5.58458425e-08 2.61956399e-02]
Predicted class is : 3
******************************************************************************
For test point: [ 4.26082551 -0.61349358]
(3, 2, 1)
(3, 2, 2)
Class 

In [135]:
# TODO [8]: Compute the accuracy of the generated Bayesian classifier 
# WITHOUT USING ANY FOR LOOPs.
accuracy = 0
# Your code should be vectorized WITHOUT USING A SINGLE FOR LOOP.
# HINT: Use np.sum and np.equal
accuracy = np.sum(np.equal(predicted_classes,Y))/Y.shape[0]
print('Accuracy = ' + str(round(accuracy,4) * 100) + '%')

Accuracy = 95.56%


In [ ]:
# TODO [9]: Generate a 3D-plot for the generated distributions. x-axis and y-axis represent the features of the data, 
#           where z-axis represent the Gaussian probability N at this point.

x = np.linspace(-10, 10, 300)
y = np.linspace(-10, 15, 300)
X, Y = np.meshgrid(x, y)
Z = np.zeros(X.shape)

for i in range(Z.shape[0]):
    for j in range(Z.shape[1]):
        # TODO [9]: Fill in the matrix Z which will represent the probability distribution of every point.
        # Z[i,j] represents the joint probability N(x,y) for x = i and y = j. 
        # We want to draw the gaussian probability N(x,y) for all points. 
        Z[i, j] = 0

# Make a 3D plot, do not change code
fig = plt.figure()
ax = fig.gca(projection='3d')
ax.plot_surface(X, Y, Z, cmap='viridis', linewidth=0)
ax.set_xlabel('X axis')
ax.set_ylabel('Y axis')
ax.set_zlabel('Z axis')
plt.show()

In [ ]:
# How can you judge your plot is correct?
'''
    Your Answer:
'''